In [19]:
import requests
import pandas as pd

from power.ml_ops.data import get_pv_data, clean_pv_data, get_data_with_cache
from power.ml_ops.model import model_yesterday
from power.ml_ops.registry import load_model
from datetime import datetime, timedelta

from pathlib import Path
from power.params import *

In [15]:
model = load_model()

data_processed_cache_path = Path(LOCAL_DATA_PATH).joinpath("processed", f"processed_pv.csv")
query = f"""
    SELECT *
    FROM {GCP_PROJECT}.{BQ_DATASET}.processed_pv
    ORDER BY utc_time
"""
data_pv_clean = get_data_with_cache(
    gcp_project=GCP_PROJECT,
    query=query,
    cache_path=data_processed_cache_path,
    data_has_header=True
    )

input_date = "2012-09-10 00:00:00"


Load latest model from local registry...

Load latest model from disk...


2024-03-09 13:43:29.003711: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-09 13:43:29.007051: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-09 13:43:29.009227: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

✅ Model loaded from local disk

Load data from local CSV...
✅ Data loaded, with shape (376944, 3)


In [ ]:
input_prediction_date = "2019-12-10 00:00:00"
# make api call
base_url = "http://127.0.0.1:8000"
# endpoint_model = "/predict/previous_value"
# endpoint_data = "/extract_data"
# url_model = f"{base_url}{endpoint_model}"
# url_data = f"{base_url}{endpoint_data}"

endpoint= "/extract_data"
# endpoint= "/predict"
url_ = f"{base_url}{endpoint}"

params ={
    'input_date':input_prediction_date
    }

# response_model = requests.get(url_model, params=params).json()
# model_df = pd.DataFrame(response_model)

# response_data = requests.get(url_data, params=params).json()
# data_df = pd.DataFrame(response_data)
response = requests.get(url_, params)
response.json()

In [41]:
def predict_baseline_yesterday(input_date: str):
    data = data_pv_clean[data_pv_clean['utc_time'] < input_date][-24:]
    values = data.electricity.to_list()
    return {input_date: values}

predict_baseline_yesterday(input_date)

{'2012-09-10 00:00:00': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.062,
  0.239,
  0.415,
  0.559,
  0.647,
  0.699,
  0.705,
  0.654,
  0.564,
  0.422,
  0.243,
  0.073,
  0.001,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]}

In [45]:
def extract_pv_data(input_date: str, n_days=10):
    """
    Returns the power data for the n_days before the input date
    Also returns the power data for the following day
    """
    n_rows = 24 * n_days
    data = data_pv_clean[data_pv_clean['utc_time'] < input_date][-n_rows:]


    # extracted_data = {
    #     'utc_time':df.get('utc_time').tolist(),
    #     'local_time':df.get('local_time').tolist(),
    #     'electricity':df.get('electricity').tolist()
    # }
    return data

extract_pv_data(input_date)

,utc_time,local_time,electricity
286344,2012-08-31 00:00:00+00:00,2012-08-31 00:00:00+00:00,0.0
286345,2012-08-31 01:00:00+00:00,2012-08-31 01:00:00+00:00,0.0
286346,2012-08-31 02:00:00+00:00,2012-08-31 02:00:00+00:00,0.0
286347,2012-08-31 03:00:00+00:00,2012-08-31 03:00:00+00:00,0.0
286348,2012-08-31 04:00:00+00:00,2012-08-31 04:00:00+00:00,0.0
...,...,...,...
286579,2012-09-09 19:00:00+00:00,2012-09-09 19:00:00+00:00,0.0
286580,2012-09-09 20:00:00+00:00,2012-09-09 20:00:00+00:00,0.0
286581,2012-09-09 21:00:00+00:00,2012-09-09 21:00:00+00:00,0.0
286582,2012-09-09 22:00:00+00:00,2012-09-09 22:00:00+00:00,0.0
